# Isl Concept

本篇主要介绍`isl`库中的一些概念, 以及一些类与函数的使用方式, 方便大家理解这些操作所代表的意义.

主要是对于[polycomp-tutorial](https://libisl.sourceforge.io/tutorial.pdf), 以及[isl manual](https://libisl.sourceforge.io/manual.pdf)做了一个粗浅总结, 建议大家尽量参考原版文档.


# Values

`isl`中使用`isl.val`表示整数值、有理值或三个特殊值之一，无穷大、负无穷大和`NaN`.可以使用以下函数创建一些预定义值.

In [2]:
import isl
from utils.common import CSource

In [3]:

print("val 构造:")

print(isl.val.one())
print(isl.val.zero())
print(isl.val.infty())
print(isl.val(123))
print(isl.val("456"))

print("val 比较:")

print(isl.val.one().is_one())
print(isl.val("1").is_one())
print(isl.val("1").is_zero())
print(isl.val("1").le(isl.val("2")))

print("val 计算:")

print(isl.val(3).mul(isl.val(2)))
print(isl.val(6).is_divisible_by(isl.val(2)))


val 构造:
1
0
infty
123
456
val 比较:
True
True
False
True
val 计算:
6
True


# Identifiers

`Identifier`用于识别单个维度和维度元组.它们由一个可选的`name`和一个可选的`void *user`组成.然而,名称和用户指针不能都是NULL.具有相同`name`但不同的`user`指针值的`id`被认为是不同的.同样,`name`不同但`user`指针值相同的标识符也被认为是不同的.相等的标识符使用相同的对象表示.因此,可以使用==运算符测试标识符对的相等性.可以使用以下函数构建、复制、释放、检查和打印标识符.

⚠️在`python`接口下, 无法随意操作指针, 所以他的`user`指针都是默认值并且会默认自增, 也无法由用户传递/修改. 因此在`python`中检查`id`等价性就少了一条标准.

In [4]:
print("Id 构造:")
a = isl.id("A")
b = isl.id("B")

print("Id 比较:")
print("构造函数内会自增指针指导致不等价:", isl.id("A") == isl.id('A'))
print(a == a)

Id 构造:
Id 比较:
构造函数内会自增指针指导致不等价: False
True


# Space

每当从头开始创建新`set`,`map`或类似对象时,都需要使用`isl_space`指定它所处的空间.每个空间涉及零个或多个参数以及集合或输入/输出维度的零\一个或两个元组.参数和维度由`isl_dim_type`和位置标识.类型`isl_dim_param`指的是参数,类型`isl_dim_set`指的是`set dimensions`(对于具有单个维度元组的空间),类型`isl_dim_in`和`isl_dim_out`指的是输入和输出维度(对于具有两个二元组的空间). `Local Space`还包含`isl_dim_div`类型的维度. 请注意,参数仅通过它们在给定对象中的位置来识别.在不同的对象中,参数(通常)通过其名称或标识符来识别.只有未命名的参数才能通过它们在对象中的位置来识别.不鼓励使用未命名的参数.

In [5]:
print("space 构造:")
unit = isl.space.unit()
print(unit)
print("添加一个参数:", unit.add_param("A"))
print("添加一个0维tuple名字为A:", unit.add_named_tuple("A", 0))
print("添加一个2维tuple名字为B:", unit.add_named_tuple("B", 2))

print("获取space :")
print("set:", isl.set("[n] -> { A[a,b,c] : a < 0 and b> 0 and c >10 }").space())
print("union_set:", isl.union_set("[n] -> { A[a,b,c]; }").space())
print("map:", isl.map("[n] -> { A[a,b,c] -> B[c,a,b] : a > 0; }").space())
print("union_map:", isl.union_map("[n] -> { A[a,b,c] -> B[c,a,b] : a > 0;  B[x] -> C[x] : x < 0;}").space())

print("space 属性:")
a = unit.add_param("A")
print("A的dim类型为IN的个数:", a.dim(isl.dim_type.IN))
print("A的dim类型为PARAM的个数:", a.dim(isl.dim_type.PARAM))

print("space 修改:")
print("找到dim中param0修改id为B:", a.set_dim_id(isl.dim_type.PARAM, 0, isl.id("B")))
sp = isl.space.unit().add_named_tuple("A", 3)
print("构造一个有名字的三个维度的tuple space:", sp)
sp = sp.set_dim_name(isl.dim_type.SET, 0, 'i')
print("修改第0个维度的名字:", sp)
print("product两个space:", sp.product(sp))


space 构造:
{  :  }
添加一个参数: [A] -> {  :  }
添加一个0维tuple名字为A: { A[] }
添加一个2维tuple名字为B: { B[i0, i1] }
获取space :
set: [n] -> { A[a, b, c] }
union_set: [n] -> {  :  }
map: [n] -> { A[a, b, c] -> B[o0, o1, o2] }
union_map: [n] -> {  :  }
space 属性:
A的dim类型为IN的个数: 0
A的dim类型为PARAM的个数: 1
space 修改:
找到dim中param0修改id为B: [B] -> {  :  }
构造一个有名字的三个维度的tuple space: { A[i0, i1, i2] }
修改第0个维度的名字: { A[i, i1, i2] }
product两个space: { [A[i, i1, i2] -> A[i', i4, i5]] }


# Sets and Relations

`polycamp`的教程中, 首先介绍了`Named Integer Tuple`, 它由命名整数元组的符号由标识符形成，然后是逗号分隔的整数列表, 比如`A[1,2,5]`. 当不设定名字时可以称为`Unnamed Integer Tuple`,比如`[2,0,4]`.

然后`set`通过花括号中使用分号分隔元素来表示, 比如`{ []; A[1,2] }`. 这在isl中被称为`union set`.


In [18]:
print("set 构造")
print(isl.set("{ A[1,2] }"))
print(isl.set("{ A[a] : a < 10 }"))
print(isl.union_set("{ []; A[1,2] }"))
print(isl.set.empty(isl.space.unit()))
print(isl.set.empty(isl.space.unit().add_named_tuple("a", 1)))

print("set 操作:")
print("intersect :", isl.union_set("{ []; A[1,2] }").intersect(isl.union_set("{ []; A[1] }")))
print("union :", isl.union_set("{ []; A[1,2] }").union(isl.union_set("{ []; C[4] }")))
print("subset :", isl.union_set("{ A[1,2] }").is_subset(isl.union_set("{  A[1,2,4] }")))
print("identity:", isl.set("{A[a] : a > 10}").identity())
print("preimage:", isl.set("{A[a] : 0 <= a < 3 }").preimage(isl.multi_aff("{ C[i] -> A[i - 10] }")))


set 构造
{ A[1, 2] }
{ A[a] : a <= 9 }
{ []; A[1, 2] }
{  : false }
{ a[i0] : false }
set 操作:
intersect : { [] }
union : { []; C[4]; A[1, 2] }
subset : False
identity: { A[a] -> A[a' = a] : a >= 11 }
preimage: { C[i] : 10 <= i <= 12 }



- Quantifier Elimination

  Quantifier Elimination是将可能存在量化变量的Presburger公式，重写为等效公式，重写后该公式不涉及任何量化的变量. 通过`compute_divs`来调用.

- Coalescing
  简化整数空间约束, 通过`coalesce`调用.

In [6]:
print("compute_divs :", isl.union_set("{ A[x] : exists a : x < 3a < 2x }").compute_divs()) 
print("coalesce :", isl.union_set("{ A[x] : exists a : x < 3a < 2x }").coalesce()) 
print("coalesce :", isl.union_set("{ A[x] : 1 <=x <= 5 or 6 <= x <= 10 }").coalesce()) 


compute_divs : { A[x] : x >= 2 and 3*floor((-1 - x)/3) > -2x }
coalesce : { A[x] : exists (e0: x < 3e0 < 2x) }
coalesce : { A[x] : 0 < x <= 10 }


## Binary Relations

在isl中使用`->`来表示两个`named integer tuple`是一个二元关系`pair`. 在`isl`中, 这种二元关系通过`union map`来表示. 并且将`pair`的前部分称为`domain`, 后部分称为`range`.

In [7]:

print("map 构造")
print(isl.union_map("{ A[2, 8, 1] -> B[5]; A[2, 8, 1] -> B[6]; B[5] -> B[5] }"))
print(isl.map("{ A[1] -> B[2,3,4]}"))
print("从domain -> range:", isl.union_map.from_domain_and_range(isl.union_set(
    "{ A[2 ,8 ,1] ; B[5]}"), isl.union_set("{ B[5]; B[6] }")))  # 这是product的构造.

print("map 操作")
print("交集:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").intersect(
    isl.union_map(" { A[2 ,8 ,1] -> B[6]; B[5] -> B[5] }")))
print("反向:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").reverse())

print("取前部分:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").range())
print("取后部分:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").domain())


map 构造
{ B[5] -> B[5]; A[2, 8, 1] -> B[6]; A[2, 8, 1] -> B[5] }
{ A[1] -> B[2, 3, 4] }
从domain -> range: { A[2, 8, 1] -> B[6]; A[2, 8, 1] -> B[5]; B[5] -> B[6]; B[5] -> B[5] }
map 操作
交集: { B[5] -> B[5] }
反向: { B[5] -> A[2, 8, 1]; B[5] -> B[5] }
取前部分: { B[5] }
取后部分: { B[5]; A[2, 8, 1] }


嵌套relation取部分：

In [8]:
wrapped = isl.map('{ [A[] -> B[]] -> [C[] -> D[]] }')
print("取前中前部分:", wrapped.domain_factor_domain())
print("取前中后部分:", wrapped.domain_factor_range())
print("取后中前部分:", wrapped.range_factor_domain())
print("取后中后部分:", wrapped.range_factor_range())

取前中前部分: { A[] -> [C[] -> D[]] }
取前中后部分: { B[] -> [C[] -> D[]] }
取后中前部分: { [A[] -> B[]] -> C[] }
取后中后部分: { [A[] -> B[]] -> D[] }


## Stride

isl中步长检测基于启发式方法。 以下函数返回的步长始终是有效的，但可能存在一些更大的有效步长未被检测到。

检查的方法是给定集合维度的值，确认是否在对某个整数值取模的情况下等于一个固定值。如果是，则将该数赋值给modulo，将该固定值赋值给 residue。如果给定维度是固定值，则将 0 赋值给 modulo，将该固定值赋值给 residue。 如果该维度并非仅取单一值，且找不到合适的模数，则将 1 赋值给 modulo，将 1 赋值给 residue。

In [9]:
domain = isl.map("{ S1[m,k] -> [m, k] : 0 <= m < 128 and 0 <= k < 256 }")
domain = domain.apply_range(isl.map('{ [m, k] -> [mo,ko,mi,ki] : mo = m mod 8 and ko = k mod 16 and mi = m // 8 and ki = k // 32 }'))
print("获得固定位置的stride:    ",domain.range().stride(3))
print("获得矩形格:              ",domain.range_lattice_tile())

获得固定位置的stride:     1
获得矩形格:               { offset: "{ S1[m, k] -> [(m), (k), (0), (0)] }", size: "{ [8, 16, 1, 1] }" }


基于上述这种stride的检测方式，对于映射规则也会有相同的结果, 可以看到下面两种不同的映射方式，获得的stride确是相同的. 但是他们的底层表示确实是不同的.

In [10]:
domainA = isl.map('{ [i] -> [i // 8, i mod 8] : 0 <= i < 128 }')
domainB = isl.map('{ [i] -> [io = i // 8, ii = i - 8*io] : 0 <= i < 128 }')
print('domainA range: ', domainA.range())
print('domainA 矩形格: ', domainA.range_lattice_tile())
print('domainB range: ', domainB.range())
print('domainB 矩形格: ', domainB.range_lattice_tile())
print('simple A     : ', domainA.domain().sample().apply(domainA))
print('simple B     : ', domainB.domain().sample().apply(domainB))

domainA range:  { [i0, i1] : 0 <= i0 <= 15 and 0 <= i1 <= 7 }
domainA 矩形格:  { offset: "{ [i] -> [(0), (i)] }", size: "{ [1, 8] }" }
domainB range:  { [io, ii] : ii >= 0 and -8io <= ii <= 127 - 8io and ii <= 7 }
domainB 矩形格:  { offset: "{ [i] -> [(0), (i)] }", size: "{ [1, 8] }" }
simple A     :  { [0, i1] : (i1) mod 8 = 0 and 0 <= i1 <= 7 }
simple B     :  { [io = 0, ii = 0] }


isl中还可以通过hull来获得凸包：

In [11]:
print("domainA 获得固定格凸包:           ", domainA.range_simple_fixed_box_hull())
print("domainB 获得固定格凸包:           ", domainB.range_simple_fixed_box_hull())
print("domain  获得固定格凸包:           ", domain.range_simple_fixed_box_hull())

print("domainA 获得affine凸包:          ", domainA.affine_hull())
print("domainB 获得affine凸包:          ", domainB.affine_hull())
print("domain  获得affine凸包:          ", domain.affine_hull())

print("domainA 获得convex凸包:          ", domainA.convex_hull())
print("domainB 获得convex凸包:          ", domainB.convex_hull())
print("domain  获得convex凸包:          ", domain.convex_hull())

print("domainA 获得polyhedral凸包:      ", domainA.polyhedral_hull())
print("domainB 获得polyhedral凸包:      ", domainB.polyhedral_hull())
print("domain  获得polyhedral凸包:      ", domain.polyhedral_hull())

print("domainA 获得simple凸包:          ", domainA.simple_hull())
print("domainB 获得simple凸包:          ", domainB.simple_hull())
print("domain  获得simple凸包:          ", domain.simple_hull())

print("domainA 获得unshifted simple凸包:", domainA.unshifted_simple_hull())
print("domainB 获得unshifted simple凸包:", domainB.unshifted_simple_hull())
print("domain  获得unshifted simple凸包:", domain.unshifted_simple_hull())

domainA 获得固定格凸包:            { offset: "{ [i] -> [(floor((i)/8)), (0)] }", size: "{ [1, 8] }" }
domainB 获得固定格凸包:            { offset: "{ [i] -> [(floor((i)/8)), (0)] }", size: "{ [1, 8] }" }
domain  获得固定格凸包:            { offset: "{ S1[m, k] -> [(0), (0), (floor((m)/8)), (floor((k)/32))] }", size: "{ [8, 16, 1, 1] }" }
domainA 获得affine凸包:           { [i] -> [o0, o1] : o1 = i - 8o0 }
domainB 获得affine凸包:           { [i] -> [io, ii] : ii = i - 8io }
domain  获得affine凸包:           { S1[m, k] -> [mo, ko, mi, ki] : 8mi = m - mo and (-k + ko) mod 16 = 0 }
domainA 获得convex凸包:           { [i] -> [o0, o1] : o1 = i - 8o0 and 0 <= i <= 127 and -7 + i <= 8o0 <= i }
domainB 获得convex凸包:           { [i] -> [io, ii] : ii = i - 8io and 0 <= i <= 127 and -7 + i <= 8io <= i }
domain  获得convex凸包:           { S1[m, k] -> [mo, ko, mi, ki] : 8mi = m - mo and (-k + ko) mod 16 = 0 and 0 <= m <= 127 and 0 <= k <= 255 and 0 <= mo <= 7 and 0 <= ko <= 15 and -31 + k <= 32ki <= k }
domainA 获得polyhedral凸包:       { [i] -

# Points

`points`是`set`的元素,通常可以用于构造`set`,或者表示`set`中所有独立的元素.

In [12]:
print("point 构造:")
unit = isl.space.unit()
unit = unit.add_param("A")
print(unit)
zero_point = isl.point.zero(unit)
print(zero_point)
print("获取一个point:", isl.set("{ A[i,j] : 0 <= i <= j < 10}").sample_point())

point 构造:
[A] -> {  :  }
[A = 0] -> {  }
获取一个point: { A[0, 0] }


In [13]:
print("point 操作:")

print("设定参数A为10:", zero_point.add_ui(isl.dim_type.PARAM, 0, 10))
isl.set("{ A[i,j] : 0 <= i <= j < 2}").foreach_point(lambda p: print(p))
print("遍历所有的point:", zero_point.set_coordinate_val(isl.dim_type.PARAM, 0, isl.val("123")))


point 操作:
设定参数A为10: [A = 10] -> {  }
{ A[0, 0] }
{ A[0, 1] }
{ A[1, 1] }
遍历所有的point: [A = 123] -> {  }


# Functions

除了`set/relation`, `isl`也提供了多种类型的函数. 每个类型都从源自`value`的类型, 或者通过零个或多个的`primitive function`类型之一来构造. 

特殊情况下, 我们也可以通过`id`来构造`multi expression`, 要注意这并不是`function`类型.

接下来我们先介绍`Primitive`类型,然后再介绍衍生类型.

## Primitive Functions

ISL支持两种`primitive function type`, 即`quasi-affine expressions`和`quasipolynomials`. 
其中`quasi-affine expression`是通过参数空间或集合定义的, 由整数常数, 参数和集合变量, 加法, 减法和整数分割组成. 这里可以参考[零基础入门多面体编译中的仿射表达式](https://zhuanlan.zhihu.com/p/627312844).

在`isl`中, `quasi-affine`描述开始于结构化的`named integer tuple`模板, 其次是`->`符号, 然后使用方括号包裹着一个使用了前面变量的表达式, 最后整个表达式包含在花括号中. 如果`quasi-affine`表达式没有`domain space`, 则可以省略前面`named integer tuple`以及`->`符号. 如果涉及任何常量参数, 则在外部继续添加参数的表示. 比如:

$$
	[n] -> { [x] -> [2*floor((4 n + x)/9)] }
$$

其中`n`是参数,`x`是变量.

`quasipolynomials`是`quasi-affine expression`中的多项式表示. 也就是它还允许乘法, 但是不允许构建涉及乘法的表达式的整数划分

$$
  [n] -> { [x] -> (n*floor((4 n + x)/9)) }
$$

In [14]:
print("quasi-affine expression :")
print(isl.aff("[n] -> { [x] -> [2*floor((4 n + x)/9)] }"))
try:
  print(isl.aff("[n] -> { [x] -> [n*floor((4 n + x)/9)] }"))
except:
  print("quasi-affine expression only expect constant value multiply ")

quasi-affine expression :
[n] -> { [x] -> [(2*floor((4n + x)/9))] }
quasi-affine expression only expect constant value multiply 


syntax error (1, 20): expecting constant value
got keyword 'floor'


### Piecewise Quasi-Affine Expression

在`isl`中，分段`quasi-affine`表达式被写成一系列有条件的aff表达式，该序列由分号分离并包裹在花括号中:


In [15]:
print("Piecewise quasi-affine expression :")
aff = isl.pw_aff("[n] -> { [x] -> [2*floor((4 n + x)/9)] :  x > 10 ; [x] -> [2*floor((4 n + x)/9)] :  x < 3   }")
print(aff, "space:", aff.get_space())
aff = isl.pw_aff("[n] -> { [x] -> [x + 1] : x < n ; [x] -> [0] : x = n - 1 }")
print(aff, "space:", aff.get_space())

Piecewise quasi-affine expression :
[n] -> { [x] -> [(2*floor((4n + x)/9))] : x >= 11; [x] -> [(2*floor((4n + x)/9))] : x <= 2 } space: [n] -> { [x] -> [o0] }
[n] -> { [x] -> [(n)] : x = -1 + n; [x] -> [(1 + x)] : x <= -2 + n } space: [n] -> { [x] -> [o0] }


如果多个`aff`的`space`不同, 那么称为`union_pw_aff`:

In [16]:
aff = isl.union_pw_aff("{ [x] -> [x+1] : x < 10;  [x,y] -> [0] }")
print(aff, "space:", aff.get_space())
aff = isl.union_pw_aff("{ [x] -> [x+1] : x < 10;  [] -> [0] }")
print(aff, "space:", aff.get_space())

{ [x] -> [(1 + x)] : x <= 9; [x, y] -> [(0)] } space: {  :  }
{ [] -> [(0)]; [x] -> [(1 + x)] : x <= 9 } space: {  :  }


`aff` 也支持各种操作:
1. `sum`, 将`space`相同的表达式的`value`求和 (使用`add`函数).
2. `union`, 使用`union add`函数.
3. `pullback`, 函数`composition`, 将一个函数的输出作用到另一个函数的输入上.

In [17]:
print("aff sum", isl.aff("{ [x] -> [x + 2] }").add(isl.aff("{ [x] -> [x - 1] }")))
print("aff union", isl.union_pw_aff(
    "{ S_0[x] -> [x + 2] }").union_add(isl.aff("{ S_1[y] -> [y - 1] }")))

a1 = isl.multi_union_pw_aff("R[{ A[i,j] -> [i + j]; E[x] -> [-x] }]")
a2 = isl.union_pw_multi_aff("{ C[x] -> A[x,x]; D[x] -> E[2x] }")
print("aff pullback:", a1.pullback(a2))


aff sum { [x] -> [(1 + 2x)] }
aff union { S_1[y] -> [(-1 + y)]; S_0[x] -> [(2 + x)] }
aff pullback: R[{ D[x] -> [(-2x)]; C[x] -> [(2x)] }]


## Reductions

`Reductions`代表其基本表达式的最大或最小值。 `isl`定义的唯一`Reductions`类型是`isl_qpolynomial_fold`。 并且无法直接创建这个类型对象, 但是可以通过`isl_pw_qpolynomial_bound`函数来返回.

## Multiple Expressions

`Multiple Expressions`表示零或更多`base expression`组成的序列, 所有`base expression`都在同一`domain space`上定义. `Multiple Expressions`的`domain space`与`base expression`相同, 但是`range space`可以是任何. 如果基本表达式具有`set space`, 相应的多个表达式也具`set space`. `value`或`identifier`的类型对象没有关联的`space`. 因此, 多个`value`或多个`identifier`的`space`始终是`set space`. 同样, `multiple union piecewise affine expression`的空间始终`set space`. 如果基本表达式不是全部的, 则相应的零维`Multiple Expressions`可能具有一个显式`domain`, 该域可以跟踪任何`base expression`之外的域.

`multiple expression` 类型包含: `isl_multi_val`, `isl_multi_id`,`isl_multi_aff`, `isl_multi_pw_aff`, `isl_multi_union_pw_aff`

In [18]:
print("multi_id 构造:")
print(isl.multi_id("{ [A,B,C] }"))
print(isl.multi_id("[D, E] -> { [A,B,C] }"))
print("multi val 构造:")
print(isl.multi_val("{ [10,20,30] }"))
print(isl.multi_val("[A, B] -> { [10,20,30] }"))

multi_id 构造:
{ [A, B, C] }
[D, E] -> { [A, B, C] }
multi val 构造:
{ [10, 20, 30] }
[A, B] -> { [10, 20, 30] }


在`isl`中, `aff`的`tuple`被表示为`isl_multi_aff`, 他的写法与`aff`类似, 但是用`[]`括起来的`aff`表达式泛化为一个结构化的命名整数元组模板(`structured named integer tuple template`), 同时这个模板的变量被替换为依赖输入命名整数元组模板变量的`aff`表达式.

In [19]:
print("isl_multi_aff 构造:")
print(isl.multi_aff("{ [i] -> [[i] -> [i+1]] }"))
print(isl.multi_aff("{ [x, y] -> [[x-2,y+1] -> [x]] }"))
print(isl.multi_aff("{ [x, y] -> [(x), (x+1)] }"))

print("isl_multi_pw_aff 构造:")
print(isl.multi_pw_aff("{ [i] -> [(i + 1: i>0), (i - 1: i < 0)]  }"))
print(isl.multi_pw_aff("{ [i] -> [[i] -> [i-1 : i < 1]]  }"))
print(isl.multi_pw_aff("{ [i] -> [[i+2] -> [(i-1 : i < 1), (i-3 : i > 1) ]] }"))

print("isl_multi_union_pw_aff 构造:")
print(isl.multi_union_pw_aff("[{ S_1[m] -> [m + 10] :  m > 0 }, { S_2[n] -> [n-10] : n < 0 } ]"))

isl_multi_aff 构造:
{ [i] -> [[(i)] -> [(1 + i)]] }
{ [x, y] -> [[(-2 + x), (1 + y)] -> [(x)]] }
{ [x, y] -> [(x), (1 + x)] }
isl_multi_pw_aff 构造:
{ [i] -> [((1 + i) : i > 0), ((-1 + i) : i < 0)] }
{ [i] -> [[(i)] -> [((-1 + i) : i <= 0)]] }
{ [i] -> [[(2 + i)] -> [((-1 + i) : i <= 0), ((-3 + i) : i >= 2)]] }
isl_multi_union_pw_aff 构造:
[{ S_1[m] -> [(10 + m)] : m > 0 }, { S_2[n] -> [(-10 + n)] : n < 0 }]


# 调度树与代码生成

首先使用ast和printer可以轻松的从`2d+1`的map表示生成到c代码。

In [20]:
builder = isl.ast_build.from_context(isl.set(' { : }'))
map = isl.union_map("{ cp2[20, t16 = 0, 0, t20 = 0, 0, t21 = 0, 0] -> [20, t16' = 0, 0, t20' = 0, 0, t21' = 0, 0]; bx[10, i, 0, j, 0, c, 0] -> [10, i' = i, 0, j' = j, 0, c' = c, 0] : 0 <= i <= 97 and 0 <= j <= 197 and 0 <= c <= 2; cp1[0, t13 = 0, 0, t18 = 0, 0, t19 = 0, 0] -> [0, t13' = 0, 0, t18' = 0, 0, t19' = 0, 0]; by[10, i, 0, j, 10, c, 0] -> [10, i' = i, 0, j' = j, 10, c' = c, 0] : 0 <= i <= 97 and 0 <= j <= 197 and 0 <= c <= 2 }")
builder = builder.node_from_schedule_map(map)

printer = isl.printer.to_file_path('/tmp/5.c')
printer = printer.set_output_format(isl.format.C)
options = isl.ast_print_options.alloc()
printer = builder.print(printer, options)
printer = printer.flush()

生成的代码如下，要注意的是这里每一个statement都是符号，具体的代码需要用户在callback中构造。
```c
{
  cp1(0, 0, 0, 0, 0, 0, 0);
  for (int c1 = 0; c1 <= 97; c1 += 1)
    for (int c3 = 0; c3 <= 197; c3 += 1) {
      for (int c5 = 0; c5 <= 2; c5 += 1)
        bx(10, c1, 0, c3, 0, c5, 0);
      for (int c5 = 0; c5 <= 2; c5 += 1)
        by(10, c1, 0, c3, 10, c5, 0);
    }
  cp2(20, 0, 0, 0, 0, 0, 0);
}
```

也可以使用schedule tree来构建ast从而生成代码，同时schedule tree就提供了更多的方式来控制ast的构造。比如我需要构造让ast进行unroll，或者分离循环迭代域。

In [21]:
domain = isl.union_set("{ S0[m,k] : 0 <= m < 1024 and 0 <= k < 512 }")
sche = isl.schedule.from_domain(domain)
partial_schedule = isl.map("{ S0[m,k] -> [k mod 8, m, k // 8 ] }")
sche = sche.insert_partial_schedule(partial_schedule.as_multi_union_pw_aff())

比如可以设置第一个循环进行展开：

In [22]:
node_band = sche.get_root().child(0)
node_band : isl.schedule_node_band
node_band = node_band.member_set_ast_loop_unroll(0)
new_sche = node_band.parent().schedule()

In [23]:
def codegen(isl_schedule: isl.schedule, i = 0):
  build = isl.ast_build.from_context(isl.set(" { : } "))
  ast_node = build.node_from(isl_schedule)
  printer = isl.printer.to_file_path(f'/tmp/{i}.c')
  printer = printer.set_output_format(isl.format.C)
  options = isl.ast_print_options.alloc()
  printer = ast_node.print(printer, options)
  printer = printer.flush()
  return CSource(f'/tmp/{i}.c')

codegen(new_sche)

```c
{
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 1);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 2);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 3);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 4);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 5);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 6);
  for (int c1 = 0; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, 8 * c2 + 7);
}

```

同时band节点还提供了另外一种设定的方式，称为isolate。 比如我可以把中间循环分离成两个部分，只需要为band node设定合适的union set，中间将来指示即可

In [24]:
node_band = sche.get_root().child(0)
node_band : isl.schedule_node_band
node_band = node_band.set_ast_build_options(isl.union_set("{ isolate[[] -> [i0, i1, i2]] : 0 <= i1 < 28 }"))
new_sche = node_band.parent().schedule()

In [25]:
codegen(new_sche)

```c
for (int c0 = 0; c0 <= 7; c0 += 1) {
  for (int c1 = 0; c1 <= 27; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, c0 + 8 * c2);
  for (int c1 = 28; c1 <= 1023; c1 += 1)
    for (int c2 = 0; c2 <= 63; c2 += 1)
      S0(c1, c0 + 8 * c2);
}

```

domain节点可以包含多个set，表示整个程序中的多个循环。这里默认构造两个循环体，他们的顺序没有强制指定：

In [26]:
domain = isl.union_set(
    "{ S1[i,j,k] : 0 <= i < 10 and 0 <= j < 10 and 0 <= k < 10; S2[l,m] : 0 <= l < 10 and 0 <= m < 10   }")

new_sche = isl.schedule.from_domain(domain)
new_sche.get_root()

isl.schedule_node_domain("""# YOU ARE HERE
domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
""")

In [27]:
codegen(new_sche)

```c
{
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      S2(c0, c1);
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      for (int c2 = 0; c2 <= 9; c2 += 1)
        S1(c0, c1, c2);
}

```

`insert_filter`是用于过滤domain中的set，这里我插入了一个新的set和一个原本有的set，最后就只能生成原本有的循环了：

In [28]:
root = new_sche.get_root()
new_node = root.child(0).insert_filter(isl.union_set("{ S3[a,b] : 0 <= a < 1024 and 0 <= b < 1024; S2[l,m] : 0 <= l < 10 and 0 <= m < 10 }"))
new_node

isl.schedule_node_filter("""domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
child:
  # YOU ARE HERE
  filter: "{ S3[a, b] : 0 <= a <= 1023 and 0 <= b <= 1023; S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
""")

In [29]:
codegen(new_node.root().schedule())

```c
for (int c0 = 0; c0 <= 9; c0 += 1)
  for (int c1 = 0; c1 <= 9; c1 += 1)
    S2(c0, c1);

```

`insert_set`实际上是插入多个filter节点，这里我插入了一个新的set和一个原本有的set两个union set，虽然中间的调度树和上面不同，但是代码生成结果相同。

In [30]:
root = new_sche.get_root()
list = isl.union_set_list(0)
list = list.add(isl.union_set("{ S3[a,b] : 0 <= a < 1024 and 0 <= b < 1024 }"))
list = list.add(isl.union_set("{ S2[l,m] : 0 <= l < 10 and 0 <= m < 10 }"))
new_node = root.child(0).insert_set(list)
new_node

isl.schedule_node_set("""domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
child:
  # YOU ARE HERE
  set:
  - filter: "{ S3[a, b] : 0 <= a <= 1023 and 0 <= b <= 1023 }"
  - filter: "{ S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
""")

In [31]:
codegen(new_node.root().schedule())

```c
for (int c0 = 0; c0 <= 9; c0 += 1)
  for (int c1 = 0; c1 <= 9; c1 += 1)
    S2(c0, c1);

```

通过`insert_sequence`方法插入序列，可以控制代码生成时的顺序：

In [32]:
root = new_sche.get_root()
list = isl.union_set_list(0)
list = list.add(isl.union_set("{ S1[i,j,k] : 0 <= i < 10 and 0 <= j < 10 and 0 <= k < 10; }"))
list = list.add(isl.union_set("{ S2[l,m] : 0 <= l < 10 and 0 <= m < 10 }"))
new_node = root.child(0).insert_sequence(list)
new_node

isl.schedule_node_sequence("""domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
child:
  # YOU ARE HERE
  sequence:
  - filter: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9 }"
  - filter: "{ S2[l, m] : 0 <= l <= 9 and 0 <= m <= 9 }"
""")

In [33]:
codegen(new_node.root().schedule())

```c
{
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      for (int c2 = 0; c2 <= 9; c2 += 1)
        S1(c0, c1, c2);
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      S2(c0, c1);
}

```

假设需要将两个statement的domain进行嵌套生成，例如我需要他们共享i，j循环，然后再分别执行S1剩余的代码， 但是使用sequence或者filter并无法做到这一点：

In [34]:
domain = isl.union_set(
    "{ S1[i,j,k] : 0 <= i < 10 and 0 <= j < 10 and 0 <= k < 10; S2[i,j] : 0 <= i < 10 and 0 <= j < 10   }")

new_sche = isl.schedule.from_domain(domain)
new_sche.get_root()

isl.schedule_node_domain("""# YOU ARE HERE
domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[i, j] : 0 <= i <= 9 and 0 <= j <= 9 }"
""")

In [35]:
codegen(new_sche)

```c
{
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      S2(c0, c1);
  for (int c0 = 0; c0 <= 9; c0 += 1)
    for (int c1 = 0; c1 <= 9; c1 += 1)
      for (int c2 = 0; c2 <= 9; c2 += 1)
        S1(c0, c1, c2);
}

```

实际上需要对domain进行time space schedule，指定他们的顺序，这样才能让他们嵌套起来：

In [36]:
root = new_sche.get_root()
band = root.child(0)
time_space_schedule = isl.union_map("{ S1[i,j,k] -> [i,j,1,k]; S2[i,j] -> [i,j,0,0] }")
new_node = band.insert_partial_schedule(time_space_schedule.as_multi_union_pw_aff())
new_node

isl.schedule_node_band("""domain: "{ S1[i, j, k] : 0 <= i <= 9 and 0 <= j <= 9 and 0 <= k <= 9; S2[i, j] : 0 <= i <= 9 and 0 <= j <= 9 }"
child:
  # YOU ARE HERE
  schedule: "[{ S2[i, j] -> [(i)]; S1[i, j, k] -> [(i)] }, { S2[i, j] -> [(j)]; S1[i, j, k] -> [(j)] }, { S2[i, j] -> [(0)]; S1[i, j, k] -> [(1)] }, { S2[i, j] -> [(0)]; S1[i, j, k] -> [(k)] }]"
""")

In [37]:
codegen(new_node.root().schedule())

```c
for (int c0 = 0; c0 <= 9; c0 += 1)
  for (int c1 = 0; c1 <= 9; c1 += 1) {
    S2(c0, c1);
    for (int c3 = 0; c3 <= 9; c3 += 1)
      S1(c0, c1, c3);
  }

```

或者可以在不改变原有调度结构情况下插入新的extension节点来执行一些其他的操作， 在插入extension节点时需要注意他的parent必须是band节点，并且执行的range需要和extension的domain匹配，这样extension才能正常工作：

In [38]:
domain = isl.union_set( "{ S1[i,j] : 0 <= i < 10 and 0 <= j < 10;  }")

new_sche = isl.schedule.from_domain(domain)
root = new_sche.get_root()
new_node = root.child(0)
new_node = root.child(0).insert_partial_schedule(isl.union_map("{ S1[i,j] -> [j,i] }").as_multi_union_pw_aff())
new_node = new_node.child(0)

extension = isl.schedule_node.from_extension(isl.union_map(
    "{ [i0, i1] -> LoadA[i0, i1]; [i0, i1] -> LoadB[i1, i0]; }"))
new_node = new_node.graft_before(extension)
new_node

isl.schedule_node_leaf("""domain: "{ S1[i, j] : 0 <= i <= 9 and 0 <= j <= 9 }"
child:
  schedule: "[{ S1[i, j] -> [(j)] }, { S1[i, j] -> [(i)] }]"
  child:
    extension: "{ [i0, i1] -> LoadA[i0, i1]; [i0, i1] -> LoadB[i1, i0] }"
    child:
      sequence:
      - filter: "{ LoadA[i0, i1]; LoadB[i0, i1] }"
      - filter: "{ S1[i, j] }"
        child:
          # YOU ARE HERE
          leaf
""")

In [39]:
codegen(new_node.root().schedule())

```c
for (int c0 = 0; c0 <= 9; c0 += 1)
  for (int c1 = 0; c1 <= 9; c1 += 1) {
    LoadA(c0, c1);
    LoadB(c1, c0);
    S1(c1, c0);
  }

```

同时extension还支持插入复杂的循环结构。这里需要注意是，由于map的domain需要与parent的range进行匹配，而isl又限制了变量只能出现在domain中，因此对于复杂的循环，需要采用wrapping的方法。 同时可以通过约束来控制当前extension的输出时机。

这个例子就展示了在特定循环位置发射一个load的循环：

In [40]:
domain = isl.union_set( "{ S1[i,j] : 0 <= i < 10 and 0 <= j < 10;  }")

new_sche = isl.schedule.from_domain(domain)
root = new_sche.get_root()
new_node = root.child(0)
new_node = root.child(0).insert_partial_schedule(isl.union_map("{ S1[i,j] -> [j,i] }").as_multi_union_pw_aff())
new_node = new_node.child(0)

extension = isl.schedule_node.from_extension(isl.union_map(
    "{ [i0, i1] -> LoadA[ Bounds[m, k] -> LocalA[i0 + m, i1 + k] ] : i0 = 0 and i1 = 2 and 0 <= m < 16 and 0 <= k < 8 }"))
new_node = new_node.graft_before(extension)
new_node

isl.schedule_node_leaf("""domain: "{ S1[i, j] : 0 <= i <= 9 and 0 <= j <= 9 }"
child:
  schedule: "[{ S1[i, j] -> [(j)] }, { S1[i, j] -> [(i)] }]"
  child:
    extension: "{ [i0 = 0, i1 = 2] -> LoadA[Bounds[m, k] -> LocalA[m, 2 + k]] : 0 <= m <= 15 and 0 <= k <= 7 }"
    child:
      sequence:
      - filter: "{ LoadA[Bounds[m, k] -> LocalA[i2, i3]] }"
      - filter: "{ S1[i, j] }"
        child:
          # YOU ARE HERE
          leaf
""")

In [41]:
codegen(new_node.root().get_schedule())

```c
for (int c0 = 0; c0 <= 9; c0 += 1)
  for (int c1 = 0; c1 <= 9; c1 += 1) {
    if (c0 == 0 && c1 == 2)
      for (int c2 = 0; c2 <= 15; c2 += 1)
        for (int c3 = 0; c3 <= 7; c3 += 1)
          LoadA(c2, c3, c2, c3 + 2);
    S1(c1, c0);
  }

```